In [461]:
import pandas as pd
import numpy as np
import random

raw_data = open('breast-cancer-wisconsin.data','r')

data=[]
result=[]
for i in raw_data:
    tem_array = i.split(',')
    tem_result = [0,0]
    if len(tem_array) == 11:
        if '?' in tem_array:
            pass
        else:
            data.append(np.array(tem_array[1:-1]).astype(float))
            tem_result[(int(tem_array[-1:][0].split('\n')[0])-2)/2]=1
            result.append( tem_result )

data = map(lambda x: x/10.0, data)
# print(len(data))
# print(len(result))

In [462]:
len_of_data = len(data)
divide_train_test = int(0.7 * len_of_data)+1

train_data = data[:divide_train_test]
test_data = data[divide_train_test:]
train_result = result[:divide_train_test]
test_result = result[divide_train_test:]

print(len(train_data))
print(len(train_result))
print(len(test_data))
print(len(test_result))

def chunks(l, n):
    for i in range(0, len(l), n):
        yield l[i:i + n]


train_data = list(chunks(train_data,50))
train_result = list(chunks(train_result,50))
len(train_result)

combined = zip(train_data,train_result)
random.shuffle(combined)

train_data[:], train_result[:] = zip(*combined)

479
479
204
204


In [463]:
import tensorflow as tf
 
# Parameters
learning_rate = 0.001
dropout = 0.65 
 
def weight_variable(shape):
    initializer = tf.truncated_normal_initializer(dtype=tf.float32, stddev=1e-1)
    return tf.get_variable("weights", shape,initializer=initializer, dtype=tf.float32)

def bias_variable(shape):
    initializer = tf.constant_initializer(0.0)
    return tf.get_variable("biases", shape, initializer=initializer, dtype=tf.float32)


original_graph = tf.Graph()
# gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.9)
sess_original = tf.Session(graph = original_graph)
with original_graph.as_default():
    
    num_class = 2

    x = tf.placeholder(tf.float32, shape=(None,9),name='x_')
    y = tf.placeholder(tf.float32, (None, num_class),name='y_')
    keep_prob = tf.placeholder(tf.float32,name='d_') #dropout (keep probability)
     

    # fc1
    with tf.variable_scope("fc1"):
        W_fc1 = weight_variable([9, 100])
        b_fc1 = bias_variable([100])
        h_fc1 = tf.nn.relu(tf.matmul(x, W_fc1) + b_fc1)

    # dropout
    h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

    # fc2
    with tf.variable_scope("fc2"):
        W_fc2 = weight_variable([100, 2])
        b_fc2 = bias_variable([2])
        y_conv = tf.Variable(0, name="y_conv")
        y_conv = tf.matmul(h_fc1_drop, W_fc2) + b_fc2
    
    ans = tf.Variable(0, name="ans")
    ans = tf.argmax(y_conv,1)
    
    
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=y_conv))

    optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss)

    correct_prediction = tf.equal(tf.argmax(y_conv,1), tf.argmax(y,1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

     
    # Initializing the variables
    #init = tf.initialize_all_variables()
    init =tf.global_variables_initializer()

    saver = tf.train.Saver()
    saver_ = tf.train.Saver()

tf.reset_default_graph()

# Launch the graph
sess_original.run(init)



In [464]:
last_acc = 0
c = 0
for _ in range(0,100):
    for i in range(0,9):
        sess_original.run(optimizer, feed_dict={x: train_data[i], y: train_result[i], keep_prob: dropout})
        acc = sess_original.run(accuracy, feed_dict={x: train_data[i], y: train_result[i], keep_prob: 1.})
        
    print(acc)
            
    if (acc - last_acc) < 0.01:
        c += 1
    last_acc = acc
    
    if c > 10:
        break

save_path = saver.save(sess, "./model/model.ckpt")
        
# #After Training, save the training status in graph
# for variable in tf.trainable_variables():
#     tensor = tf.constant(variable.eval())
#     tf.assign(variable, tensor, name="nWeights")
#     #Save graph
# tf.train.write_graph(sess_original.graph_def, 'model/', 'graph_vsd.pb', as_text=False)
# print ("save graph over")

0.48
0.48
0.7
0.8
0.84
0.88
0.94
0.94
0.94
0.92
0.92
0.94
0.94
0.94
0.94
0.94
0.94
0.94


In [465]:
acc = sess_original.run(accuracy, feed_dict={x: test_data, y: test_result, keep_prob: 1.})
y = sess_original.run(ans, feed_dict={ x: test_data, keep_prob: 1. })
print(acc)
saver.save(sess_original, 'model/model',global_step=0)
tf.train.write_graph(sess_original.graph_def, 'model/', 'graph_vsd.pb', as_text=False)  


0.995098


'model/graph_vsd.pb'

In [466]:
y

array([1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0,
       0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0,
       1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1,
       0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1])

In [2]:
import numpy as np  
X = np.array([[-1, -1], [-2, -1], [-3, -2], [1, 1], [2, 1], [3, 2]])  
Y = np.array([1, 2, 3, 4, 5, 6])  
from sklearn.naive_bayes import GaussianNB  

ImportError: No module named sklearn.naive_bayes